In [185]:
#!/usr/bin/env python
# coding: utf-8

import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

import json
import time

import seisbench.models as sbm
import seisbench.data as sbd
import seisbench.generate as sbg

from seisbench.util import worker_seeding
from torch.utils.data import DataLoader
from obspy.clients.fdsn import Client
from scipy.signal import find_peaks



In [186]:
print("✓ All imports successful!")

✓ All imports successful!


In [187]:
# Load configuration from JSON file
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    print("✓ Configuration loaded successfully!")
except FileNotFoundError:
    print("✗ Error: config.json file not found!")
    raise
except json.JSONDecodeError as e:
    print(f"✗ Error: Invalid JSON in config.json: {e}")
    raise
except Exception as e:
    print(f"✗ Unexpected error loading config: {e}")
    raise

✓ Configuration loaded successfully!


In [188]:
config

{'peak_detection': {'sampling_rate': 100, 'height': 0.5, 'distance': 100},
 'training': {'batch_size': 64,
  'num_workers': 4,
  'learning_rate': 0.01,
  'epochs': 50,
  'patience': 5,
  'loss_weights': [0.01, 0.4, 0.59],
  'optimization': {'mixed_precision': True,
   'gradient_accumulation_steps': 1,
   'pin_memory': True,
   'prefetch_factor': 2,
   'persistent_workers': True}},
 'device': {'use_cuda': True, 'device_id': 0}}

In [189]:
# Set random seed for reproducibility
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [190]:
import os

# Loader the picker
try:
    #model = sbm.EQTransformer.from_pretrained("original")
    model = sbm.PhaseNet.from_pretrained("stead")
    print("✓ Model loaded successfully!")
except Exception as e:
    print(f"✗ Error loading model: {e}")
    raise

✓ Model loaded successfully!


In [191]:
# Set up device
device = torch.device(f"cuda:{config['device']['device_id']}" if torch.cuda.is_available() and config['device']['use_cuda'] else "cpu")
print(f"Using device: {device}")
model.to(device)


Using device: cpu


PhaseNet(
  (inc): Conv1d(3, 8, kernel_size=(7,), stride=(1,), padding=same)
  (in_bn): BatchNorm1d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (down_branch): ModuleList(
    (0): ModuleList(
      (0): Conv1d(8, 8, kernel_size=(7,), stride=(1,), padding=same, bias=False)
      (1): BatchNorm1d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv1d(8, 8, kernel_size=(7,), stride=(4,), padding=(3,), bias=False)
      (3): BatchNorm1d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ModuleList(
      (0): Conv1d(8, 16, kernel_size=(7,), stride=(1,), padding=same, bias=False)
      (1): BatchNorm1d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv1d(16, 16, kernel_size=(7,), stride=(4,), bias=False)
      (3): BatchNorm1d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): ModuleList(
      (0): Conv1d(16, 32, kernel_size=(7,), stride=(1,

In [192]:
# Print PhaseNet model information
phasenet_total_params = sum(p.numel() for p in model.parameters())
phasenet_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("PhaseNet information:")
print(f"Total parameters: {phasenet_total_params:,}")
print(f"Trainable parameters: {phasenet_trainable_params:,}")
print(f"Model size: {phasenet_total_params * 4 / (1024**2):.2f} MB (float32)")

PhaseNet information:
Total parameters: 268,443
Trainable parameters: 268,443
Model size: 1.02 MB (float32)


In [193]:
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")# Display all PhaseNet model parameters with their names and shapes
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

inc.weight: torch.Size([8, 3, 7])
inc.bias: torch.Size([8])
in_bn.weight: torch.Size([8])
in_bn.bias: torch.Size([8])
down_branch.0.0.weight: torch.Size([8, 8, 7])
down_branch.0.1.weight: torch.Size([8])
down_branch.0.1.bias: torch.Size([8])
down_branch.0.2.weight: torch.Size([8, 8, 7])
down_branch.0.3.weight: torch.Size([8])
down_branch.0.3.bias: torch.Size([8])
down_branch.1.0.weight: torch.Size([16, 8, 7])
down_branch.1.1.weight: torch.Size([16])
down_branch.1.1.bias: torch.Size([16])
down_branch.1.2.weight: torch.Size([16, 16, 7])
down_branch.1.3.weight: torch.Size([16])
down_branch.1.3.bias: torch.Size([16])
down_branch.2.0.weight: torch.Size([32, 16, 7])
down_branch.2.1.weight: torch.Size([32])
down_branch.2.1.bias: torch.Size([32])
down_branch.2.2.weight: torch.Size([32, 32, 7])
down_branch.2.3.weight: torch.Size([32])
down_branch.2.3.bias: torch.Size([32])
down_branch.3.0.weight: torch.Size([64, 32, 7])
down_branch.3.1.weight: torch.Size([64])
down_branch.3.1.bias: torch.Size([

In [194]:
# Import XiaoNet from models
import sys
sys.path.append('/Users/hongyuxiao/Hongyu_File/xiao_net')
from models.xn_xiao_net import XiaoNet

# Create XiaoNet model
xiao_net = XiaoNet(
    window_len=3001,      # Match the data window length
    in_channels=3,        # 3 channels (E, N, Z)
    num_phases=3,         # 3 outputs (P, S, noise)
    base_channels=16      # Base channel width (can adjust for size)
)
xiao_net.to(device)

# Count parameters
total_params = sum(p.numel() for p in xiao_net.parameters())
trainable_params = sum(p.numel() for p in xiao_net.parameters() if p.requires_grad)

print(f"✓ XiaoNet created successfully!")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: {total_params * 4 / (1024**2):.2f} MB (float32)")
print(f"Base channels: 16")
print(f"Kernel size: 3 (all convolution layers)")

✓ XiaoNet created successfully!
Total parameters: 164,355
Trainable parameters: 164,355
Model size: 0.63 MB (float32)
Base channels: 16
Kernel size: 3 (all convolution layers)


In [195]:
# Test XiaoNet forward pass with 3001 input to debug dimension flow
import importlib
import sys

# Remove old module from cache if it exists
if 'models.xn_xiao_net' in sys.modules:
    del sys.modules['models.xn_xiao_net']

# Re-import with fresh code
from models.xn_xiao_net import XiaoNet

# Create XiaoNet model with fixed forward pass
xiao_net = XiaoNet(
    window_len=3001,      # Match the data window length
    in_channels=3,        # 3 channels (E, N, Z)
    num_phases=3,         # 3 outputs (P, S, noise)
    base_channels=16      # Base channel width (can adjust for size)
)
xiao_net.to(device)

# Test with a dummy batch to verify dimensions
test_input = torch.randn(2, 3, 3001).to(device)
try:
    test_output = xiao_net(test_input)
    print(f"✓ XiaoNet forward pass successful!")
    print(f"  Input shape: {test_input.shape}")
    print(f"  Output shape: {test_output.shape}")
except Exception as e:
    print(f"✗ Error during forward pass: {e}")
    raise

# Count parameters
total_params = sum(p.numel() for p in xiao_net.parameters())
trainable_params = sum(p.numel() for p in xiao_net.parameters() if p.requires_grad)

print(f"\n✓ XiaoNet reloaded with fixed forward pass!")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: {total_params * 4 / (1024**2):.2f} MB (float32)")
print(f"Base channels: 16")
print(f"Kernel size: 3 (all convolution layers)")


✓ XiaoNet forward pass successful!
  Input shape: torch.Size([2, 3, 3001])
  Output shape: torch.Size([2, 3, 3001])

✓ XiaoNet reloaded with fixed forward pass!
Total parameters: 164,355
Trainable parameters: 164,355
Model size: 0.63 MB (float32)
Base channels: 16
Kernel size: 3 (all convolution layers)


In [196]:
# Load the data
print("Loading data...")
try:
    data = sbd.OKLA_1Mil_120s_Ver_3(sampling_rate=100, force=True, component_order="ENZ")
    print("✓ Data loaded successfully!")

except Exception as e:

    print(f"✗ Error loading data: {e}")
    raise

Loading data...
✓ Data loaded successfully!


In [197]:
# Create a random sample
sample_fraction = 0.02  # Sample 20% of the data
print(f"Creating random sample of {sample_fraction*100}% of the data...")

Creating random sample of 2.0% of the data...


In [198]:
# Create a random mask for sampling
np.random.seed(42)  # For reproducibility

mask = np.random.random(len(data)) < sample_fraction
data.filter(mask)

print(f"Sampled dataset size: {len(data)}")


Sampled dataset size: 22782


In [199]:
print("Sample metadata:")
data.metadata.head(20)

Sample metadata:


,index,station_network_code,station_code,trace_channel,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_p_arrival_sample,trace_p_status,trace_p_weight,...,trace_snr_db,trace_coda_end_sample,trace_start_time,trace_category,trace_name,split,trace_name_original,trace_chunk,trace_sampling_rate_hz,trace_component_order
72,72,2V,TG11,EHE,35.2689,-97.8146,407.0,6000.0,manual,1.0,...,NaN,NaN,2023-09-11T10:04:26.195000Z,earthquake_local,"bucket0$53,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2023-09-11T1004262023-09-1...,,100,ZNE
128,128,2V,TG11,EHE,35.2689,-97.8146,407.0,6000.0,manual,1.0,...,NaN,NaN,2023-10-15T03:02:11.464999Z,earthquake_local,"bucket0$97,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2023-10-15T0302112023-10-1...,,100,ZNE
171,171,2V,TG11,EHE,35.2689,-97.8146,407.0,6000.0,manual,1.0,...,NaN,NaN,2023-11-02T09:08:31.285000Z,earthquake_local,"bucket0$124,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2023-11-02T0908312023-11-0...,,100,ZNE
205,205,2V,TG11,EHE,35.2689,-97.8146,407.0,6000.0,manual,1.0,...,NaN,NaN,2023-11-29T04:45:18.075000Z,earthquake_local,"bucket5$28,:3,:12001",test,2V.TG11.EHE.EHN.EHZ.2023-11-29T0445182023-11-2...,,100,ZNE
208,208,2V,TG11,EHE,35.2689,-97.8146,407.0,5999.0,manual,1.0,...,NaN,NaN,2023-11-30T00:58:44.934999Z,earthquake_local,"bucket0$153,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2023-11-30T0058442023-11-3...,,100,ZNE
244,244,2V,TG11,EHE,35.2689,-97.8146,407.0,5999.0,manual,1.0,...,NaN,NaN,2023-12-13T15:05:12.134999Z,earthquake_local,"bucket0$180,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2023-12-13T1505122023-12-1...,,100,ZNE
332,332,2V,TG11,EHE,35.2689,-97.8146,407.0,6000.0,manual,1.0,...,NaN,NaN,2024-01-20T00:11:37.395000Z,earthquake_local,"bucket0$239,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2024-01-20T0011372024-01-2...,,100,ZNE
371,371,2V,TG11,EHE,35.2689,-97.8146,407.0,5999.0,manual,1.0,...,NaN,NaN,2024-01-24T07:04:50.404999Z,earthquake_local,"bucket4$46,:3,:12001",dev,2V.TG11.EHE.EHN.EHZ.2024-01-24T0704502024-01-2...,,100,ZNE
407,407,2V,TG11,EHE,35.2689,-97.8146,407.0,5999.0,manual,1.0,...,NaN,NaN,2024-02-03T09:44:29.134999Z,earthquake_local,"bucket0$294,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2024-02-03T0944292024-02-0...,,100,ZNE
456,456,2V,TG11,EHE,35.2689,-97.8146,407.0,6000.0,manual,1.0,...,NaN,NaN,2024-02-12T03:21:08.685000Z,earthquake_local,"bucket0$328,:3,:12001",train,2V.TG11.EHE.EHN.EHZ.2024-02-12T0321082024-02-1...,,100,ZNE


In [200]:
# Split data
train, dev, test = data.train_dev_test()

In [201]:

print("Train:", train)
print("Dev:", dev)
print("Test:", test)


Train: OKLA_1Mil_120s_Ver_3 - 15909 traces
Dev: OKLA_1Mil_120s_Ver_3 - 3429 traces
Test: OKLA_1Mil_120s_Ver_3 - 3444 traces


In [202]:
# Set up data augmentation

phase_dict = {
    "trace_p_arrival_sample": "P",
    "trace_pP_arrival_sample": "P",
    "trace_P_arrival_sample": "P",
    "trace_P1_arrival_sample": "P",
    "trace_Pg_arrival_sample": "P",
    "trace_Pn_arrival_sample": "P",
    "trace_PmP_arrival_sample": "P",
    "trace_pwP_arrival_sample": "P",
    "trace_pwPm_arrival_sample": "P",
    "trace_s_arrival_sample": "S",
    "trace_S_arrival_sample": "S",
    "trace_S1_arrival_sample": "S",
    "trace_Sg_arrival_sample": "S",
    "trace_SmS_arrival_sample": "S",
    "trace_Sn_arrival_sample": "S",
}

In [203]:
# Create the data generators for training and validation
train_generator = sbg.GenericGenerator(train)
dev_generator = sbg.GenericGenerator(dev)
test_generator = sbg.GenericGenerator(test)

In [204]:
# Define phase lists for labeling
p_phases = [key for key, val in phase_dict.items() if val == "P"]
s_phases = [key for key, val in phase_dict.items() if val == "S"]

train_generator = sbg.GenericGenerator(train)
dev_generator = sbg.GenericGenerator(dev)
test_generator = sbg.GenericGenerator(test)

augmentations = [
    sbg.WindowAroundSample(list(phase_dict.keys()), samples_before=3000, windowlen=6000, selection="random", strategy="variable"),
    sbg.RandomWindow(windowlen=3001, strategy="pad"),
    sbg.Normalize(demean_axis=-1, detrend_axis=-1, amp_norm_axis=-1, amp_norm_type="peak"),
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(sigma=30, dim=0),
]

train_generator.add_augmentations(augmentations)
dev_generator.add_augmentations(augmentations)
test_generator.add_augmentations(augmentations)


In [205]:
# Parameters for peak detection
sampling_rate = config['peak_detection']['sampling_rate']
height = config['peak_detection']['height']
distance = config['peak_detection']['distance']

batch_size = config['training']['batch_size']
num_workers = config['training']['num_workers']

In [206]:
print("=" * 60)
print("TRAINING CONFIGURATION SANITY CHECK")
print("=" * 60)

print("\n📊 Dataset Information:")
print(f"  Total dataset size: {len(data):,} samples")
print(f"  Training set: {len(train):,} samples")
print(f"  Validation set: {len(dev):,} samples")
print(f"  Test set: {len(test):,} samples")
print(f"  Sample fraction used: {sample_fraction*100}%")

print("\n🔧 Data Augmentation Pipeline:")
for i, aug in enumerate(augmentations, 1):
    print(f"  {i}. {aug.__class__.__name__}")
    if hasattr(aug, '__dict__'):
        relevant_attrs = {k: v for k, v in aug.__dict__.items() if not k.startswith('_')}
        for key, val in relevant_attrs.items():
            print(f"     - {key}: {val}")

print("\n🎯 Model Information:")
print(f"  Model type: {model.__class__.__name__}")
print(f"  Total parameters: {phasenet_total_params:,}")
print(f"  Trainable parameters: {phasenet_trainable_params:,}")
print(f"  Model size: {phasenet_total_params * 4 / (1024**2):.2f} MB")
print(f"  Device: {device}")

print("\n⚙️ Training Hyperparameters:")
print(f"  Batch size: {batch_size}")
print(f"  Number of workers: {num_workers}")
print(f"  Learning rate: {config['training']['learning_rate']}")
print(f"  Number of epochs: {config['training']['epochs']}")
print(f"  Patience (early stopping): {config['training']['patience']}")

print("\n📈 Peak Detection Parameters:")
print(f"  Sampling rate: {sampling_rate} Hz")
print(f"  Height threshold: {height}")
print(f"  Distance: {distance} samples")

print("\n🔬 Phase Dictionary:")
print(f"  P-phases: {len(p_phases)} types")
print(f"  S-phases: {len(s_phases)} types")

print("\n💾 Data Loader Configuration:")
print(f"  Pin memory: True")
print(f"  Prefetch factor: 4")
print(f"  Persistent workers: True")

print("\n" + "=" * 60)
print("Ready to start training!")
print("=" * 60)

TRAINING CONFIGURATION SANITY CHECK

📊 Dataset Information:
  Total dataset size: 22,782 samples
  Training set: 15,909 samples
  Validation set: 3,429 samples
  Test set: 3,444 samples
  Sample fraction used: 2.0%

🔧 Data Augmentation Pipeline:
  1. WindowAroundSample
     - metadata_keys: ['trace_p_arrival_sample', 'trace_pP_arrival_sample', 'trace_P_arrival_sample', 'trace_P1_arrival_sample', 'trace_Pg_arrival_sample', 'trace_Pn_arrival_sample', 'trace_PmP_arrival_sample', 'trace_pwP_arrival_sample', 'trace_pwPm_arrival_sample', 'trace_s_arrival_sample', 'trace_S_arrival_sample', 'trace_S1_arrival_sample', 'trace_Sg_arrival_sample', 'trace_SmS_arrival_sample', 'trace_Sn_arrival_sample']
     - samples_before: 3000
     - selection: random
     - p0: None
     - windowlen: 6000
     - strategy: variable
     - axis: -1
     - key: ('X', 'X')
  2. RandomWindow
     - p0: None
     - windowlen: 3001
     - strategy: pad
     - axis: -1
     - key: ('X', 'X')
     - low: None
     - hig

In [207]:
# Load the data for machine learning

train_loader = DataLoader(train_generator,batch_size=batch_size, shuffle=True, num_workers=num_workers, worker_init_fn=worker_seeding,pin_memory=True,prefetch_factor=4,persistent_workers=True)
test_loader = DataLoader(test_generator, batch_size=batch_size, shuffle=False, num_workers=num_workers, worker_init_fn=worker_seeding,pin_memory=True,prefetch_factor=4,persistent_workers=True)
val_loader = DataLoader(dev_generator, batch_size=batch_size, shuffle=False, num_workers=num_workers, worker_init_fn=worker_seeding,pin_memory=True,prefetch_factor=4,persistent_workers=True)


In [208]:
# Define loss function
def loss_fn(y_pred, y_true, eps=1e-5):
    h = y_true * torch.log(y_pred + eps)
    h = h.mean(-1).sum(-1)
    h = h.mean()
    return -h

In [209]:
# Knowledge Distillation Loss Function
def distillation_loss_fn(student_pred, teacher_pred, y_true, temperature=3.0, alpha=0.5, eps=1e-5):
    """
    Combined loss for knowledge distillation.
    
    Args:
        student_pred: Student model predictions (after softmax)
        teacher_pred: Teacher model predictions (after softmax)
        y_true: Ground truth labels
        temperature: Temperature for softening probability distributions (higher = softer)
        alpha: Weight between distillation loss and task loss (0-1)
               alpha=1.0: only learn from teacher
               alpha=0.0: only learn from ground truth
        eps: Small epsilon for numerical stability
    
    Returns:
        Combined distillation loss
    """
    # Task loss: student vs ground truth (standard cross-entropy)
    task_loss = -(y_true * torch.log(student_pred + eps)).mean(-1).sum(-1).mean()
    
    # Distillation loss: student vs teacher (KL divergence with temperature scaling)
    # Apply temperature scaling to soften the distributions
    student_log_soft = torch.log((student_pred ** (1.0/temperature)) + eps)
    teacher_soft = teacher_pred ** (1.0/temperature)
    
    # Normalize after temperature scaling
    student_log_soft = student_log_soft - torch.logsumexp(student_log_soft, dim=1, keepdim=True)
    teacher_soft = teacher_soft / teacher_soft.sum(dim=1, keepdim=True)
    
    # KL divergence
    distill_loss = -(teacher_soft * student_log_soft).mean(-1).sum(-1).mean()
    
    # Scale distillation loss by T^2 (common practice in knowledge distillation)
    distill_loss = distill_loss * (temperature ** 2)
    
    # Combine losses
    total_loss = alpha * distill_loss + (1 - alpha) * task_loss
    
    return total_loss

print("✓ Knowledge distillation loss function defined!")
print(f"  - Temperature: Controls softness of probability distributions")
print(f"  - Alpha: Balances learning from teacher vs ground truth")

✓ Knowledge distillation loss function defined!
  - Temperature: Controls softness of probability distributions
  - Alpha: Balances learning from teacher vs ground truth


In [210]:
# Configure Teacher and Student Models for Knowledge Distillation
print("=" * 60)
print("KNOWLEDGE DISTILLATION SETUP")
print("=" * 60)

# Teacher model: PhaseNet (pretrained, frozen)
teacher_model = model  # PhaseNet loaded earlier
teacher_model.eval()   # Set to evaluation mode

# Freeze all teacher parameters - no training
for param in teacher_model.parameters():
    param.requires_grad = False

teacher_params = sum(p.numel() for p in teacher_model.parameters())
teacher_trainable = sum(p.numel() for p in teacher_model.parameters() if p.requires_grad)

print("\n👨‍🏫 Teacher Model (PhaseNet):")
print(f"  Total parameters: {teacher_params:,}")
print(f"  Trainable parameters: {teacher_trainable:,} (frozen ✓)")
print(f"  Model size: {teacher_params * 4 / (1024**2):.2f} MB")

# Student model: XiaoNet (will be trained)
student_model = xiao_net  # XiaoNet created earlier
student_model.train()      # Set to training mode

student_params = sum(p.numel() for p in student_model.parameters())
student_trainable = sum(p.numel() for p in student_model.parameters() if p.requires_grad)

print("\n👨‍🎓 Student Model (XiaoNet):")
print(f"  Total parameters: {student_params:,}")
print(f"  Trainable parameters: {student_trainable:,}")
print(f"  Model size: {student_params * 4 / (1024**2):.2f} MB")

print("\n📊 Model Comparison:")
print(f"  Parameter reduction: {(1 - student_params/teacher_params)*100:.1f}%")
print(f"  Size reduction: {(1 - (student_params * 4 / (1024**2)) / (teacher_params * 4 / (1024**2)))*100:.1f}%")

print("\n" + "=" * 60)
print("✓ Teacher-Student configuration complete!")
print("=" * 60)

KNOWLEDGE DISTILLATION SETUP

👨‍🏫 Teacher Model (PhaseNet):
  Total parameters: 268,443
  Trainable parameters: 0 (frozen ✓)
  Model size: 1.02 MB

👨‍🎓 Student Model (XiaoNet):
  Total parameters: 164,355
  Trainable parameters: 164,355
  Model size: 0.63 MB

📊 Model Comparison:
  Parameter reduction: 38.8%
  Size reduction: 38.8%

✓ Teacher-Student configuration complete!


In [211]:
# Learning rate and number of epochs
learning_rate = config['training']['learning_rate']
epochs = config['training']['epochs']


In [212]:
# Setup optimizer for STUDENT model only (teacher is frozen)
optimizer = torch.optim.Adam(student_model.parameters(), lr=learning_rate)

print(f"✓ Optimizer configured for student model")
print(f"  Learning rate: {learning_rate}")
print(f"  Optimizing {student_trainable:,} parameters")


✓ Optimizer configured for student model
  Learning rate: 0.01
  Optimizing 164,355 parameters


In [213]:
# Training function with knowledge distillation
def train_one_epoch_distillation(dataloader, student_model, teacher_model, optimizer, device, 
                                   temperature=3.0, alpha=0.5):
    """
    Train student model for one epoch using knowledge distillation.
    
    Args:
        dataloader: Training data loader
        student_model: Student model to train
        teacher_model: Teacher model (frozen)
        optimizer: Optimizer for student model
        device: Device to train on
        temperature: Temperature for distillation
        alpha: Weight between distillation and task loss
    """
    student_model.train()
    teacher_model.eval()  # Teacher always in eval mode
    
    total_loss = 0
    task_loss_total = 0
    distill_loss_total = 0
    size = len(dataloader.dataset)

    for batch_id, batch in enumerate(dataloader):
        X = batch["X"].to(device)
        y_true = batch["y"].to(device)
        
        # Get predictions from both models
        student_pred = student_model(X)
        
        with torch.no_grad():  # No gradients for teacher
            teacher_pred = teacher_model(X)
        
        # Calculate combined loss
        loss = distillation_loss_fn(student_pred, teacher_pred, y_true, 
                                     temperature=temperature, alpha=alpha)
        
        # Also calculate individual losses for monitoring
        eps = 1e-5
        task_loss = -(y_true * torch.log(student_pred + eps)).mean(-1).sum(-1).mean()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_id % 5 == 0:
            print(f"loss: {loss.item():>7f}  [{batch_id * len(batch['X']):>5d}/{size:>5d}]")

        total_loss += loss.item()
        task_loss_total += task_loss.item()

    avg_loss = total_loss / len(dataloader)
    avg_task_loss = task_loss_total / len(dataloader)
    
    return avg_loss, avg_task_loss


# Evaluation function for student model
def evaluate_student_model(dataloader, student_model, device):
    """
    Evaluate student model on validation/test set.
    Uses standard cross-entropy loss (no teacher involved).
    """
    student_model.eval()
    val_loss = 0
    eps = 1e-5

    with torch.no_grad():
        for batch in dataloader:
            X = batch["X"].to(device)
            y_true = batch["y"].to(device)
            
            student_pred = student_model(X)
            loss = -(y_true * torch.log(student_pred + eps)).mean(-1).sum(-1).mean()
            val_loss += loss.item()

    return val_loss / len(dataloader)


print("✓ Distillation training functions defined!")
print("  - train_one_epoch_distillation: Trains student with teacher guidance")
print("  - evaluate_student_model: Evaluates student independently")

✓ Distillation training functions defined!
  - train_one_epoch_distillation: Trains student with teacher guidance
  - evaluate_student_model: Evaluates student independently


In [214]:
# EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, checkpoint_path='checkpoint.pt', 
                 best_model_path='best_model.pth', final_model_path='final_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.checkpoint_path = checkpoint_path
        self.best_model_path = best_model_path
        self.final_model_path = final_model_path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.save_best_model(model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                self.save_final_model(model)
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.save_best_model(model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.checkpoint_path)
        self.val_loss_min = val_loss

    def save_best_model(self, model):
        if self.verbose:
            print(f'Saving best model to {self.best_model_path}')
        torch.save(model.state_dict(), self.best_model_path)

    def save_final_model(self, model):
        if self.verbose:
            print(f'Early stopping triggered. Saving final model to {self.final_model_path}')
        torch.save(model.state_dict(), self.final_model_path)


In [215]:
# Function to train for one epoch
def train_one_epoch(dataloader, model, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    size = len(dataloader.dataset)

    for batch_id, batch in enumerate(dataloader):
        pred = model(batch["X"].to(device))
        loss = loss_fn(pred, batch["y"].to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_id % 5 == 0:
            print(f"loss: {loss.item():>7f}  [{batch_id * len(batch['X']):>5d}/{size:>5d}]")

        total_loss += loss.item()

    return total_loss / len(dataloader)


In [216]:
# Function to evaluate the model
def evaluate_model(dataloader, model, loss_fn, device):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            pred = model(batch["X"].to(device))
            val_loss += loss_fn(pred, batch["y"].to(device)).item()

    return val_loss / len(dataloader)


In [217]:
# Function to plot training history
def plot_training_history(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Time')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.fill_between(range(len(history['train_loss'])), 
                     history['train_loss'], history['val_loss'],
                     alpha=0.3, color='red', 
                     where=(np.array(history['val_loss']) > np.array(history['train_loss'])),
                     label='Potential Overfitting Gap')
    plt.savefig('training_history.png')
    plt.close()


In [ ]:
# Training routine with EarlyStopping and scheduler for Knowledge Distillation
def train_model_distillation(train_loader, val_loader, student_model, teacher_model, optimizer, device, 
                              num_epochs=25, patience=7, temperature=3.0, alpha=0.5):
    """
    Train student model using knowledge distillation with EarlyStopping and scheduler.
    
    Args:
        train_loader: Training data loader
        val_loader: Validation data loader
        student_model: Student model to train
        teacher_model: Teacher model (frozen)
        optimizer: Optimizer for student model
        device: Device to train on
        num_epochs: Maximum number of epochs
        patience: EarlyStopping patience
        temperature: Temperature for softening probability distributions
        alpha: Weight between distillation and task loss
    """
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    history = {
        'train_loss': [],
        'val_loss': []
    }

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("=" * 60)
        
        # Training phase (distillation)
        train_loss, task_loss = train_one_epoch_distillation(
            train_loader, student_model, teacher_model, optimizer, device, 
            temperature=temperature, alpha=alpha
        )
        
        # Validation phase (student only)
        val_loss = evaluate_student_model(val_loader, student_model, device)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Train Loss: {train_loss:.6f}")
        print(f"  Train Task Loss: {task_loss:.6f}")
        print(f"  Val Loss: {val_loss:.6f}")

        scheduler.step(val_loss)
        early_stopping(val_loss, student_model)
        if early_stopping.early_stop:
            print("\n" + "=" * 60)
            print("✓ Early stopping triggered!")
            break

    # Load best model
    student_model.load_state_dict(torch.load('checkpoint.pt'))
    plot_training_history(history)
    return student_model, history

if __name__ == "__main__":
    # Call the training function with distillation
    patience = config['training']['patience'] if 'patience' in config['training'] else 7
    
    trained_student, training_history = train_model_distillation(
        train_loader=train_loader,
        val_loader=val_loader,
        student_model=student_model,
        teacher_model=teacher_model,
        optimizer=optimizer,
        device=device,
        num_epochs=epochs,
        patience=patience,
        temperature=3.0,
        alpha=0.5
    )
    
    # Evaluate student on test set
    test_loss = evaluate_student_model(test_loader, trained_student, device)
    print(f"\n{'='*60}")
    print(f"Final Test Loss: {test_loss:.6f}")
    print(f"{'='*60}")
    
    print("\n✓ Knowledge Distillation Training Completed!")



Epoch 1/50


/opt/anaconda3/envs/ML/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


loss: 6.131898  [    0/15909]
loss: 4.966187  [  320/15909]
loss: 4.228575  [  640/15909]
loss: 3.522102  [  960/15909]
loss: 2.851644  [ 1280/15909]
loss: 2.246575  [ 1600/15909]
loss: 1.879482  [ 1920/15909]
loss: 1.530304  [ 2240/15909]
loss: 1.407449  [ 2560/15909]
loss: 1.323392  [ 2880/15909]
loss: 1.295061  [ 3200/15909]
loss: 1.224062  [ 3520/15909]
loss: 1.192209  [ 3840/15909]
loss: 1.254235  [ 4160/15909]
loss: 1.225917  [ 4480/15909]
loss: 1.239346  [ 4800/15909]
loss: 1.247718  [ 5120/15909]
loss: 1.253661  [ 5440/15909]
loss: 1.204089  [ 5760/15909]
loss: 1.220528  [ 6080/15909]
loss: 1.209281  [ 6400/15909]
loss: 1.196747  [ 6720/15909]
loss: 1.142924  [ 7040/15909]
loss: 1.160721  [ 7360/15909]
loss: 1.197099  [ 7680/15909]
loss: 1.153111  [ 8000/15909]
loss: 1.172551  [ 8320/15909]
loss: 1.117908  [ 8640/15909]
loss: 1.065643  [ 8960/15909]
loss: 1.108493  [ 9280/15909]
loss: 1.104210  [ 9600/15909]
loss: 1.224569  [ 9920/15909]
loss: 1.043436  [10240/15909]
loss: 1.06

: 

In [2]:
# Comprehensive Teacher vs Student Model Evaluation
def evaluate_model_comprehensive(dataloader, model, model_name, device):
    """
    Comprehensive evaluation of a model on a dataset.
    
    Returns:
        dict with loss, accuracy metrics, and predictions
    """
    model.eval()
    val_loss = 0
    eps = 1e-5
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            X = batch["X"].to(device)
            y_true = batch["y"].to(device)
            
            pred = model(X)
            loss = -(y_true * torch.log(pred + eps)).mean(-1).sum(-1).mean()
            val_loss += loss.item()
            
            all_preds.append(pred.cpu())
            all_labels.append(y_true.cpu())
    
    avg_loss = val_loss / len(dataloader)
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    
    # Calculate accuracy metrics
    pred_classes = torch.argmax(all_preds, dim=1)  # (batch, time)
    label_classes = torch.argmax(all_labels, dim=1)  # (batch, time)
    
    accuracy = (pred_classes == label_classes).float().mean().item()
    
    # Per-class accuracy
    class_names = ['P-wave', 'S-wave', 'Noise']
    per_class_acc = {}
    
    for i, class_name in enumerate(class_names):
        mask = (label_classes == i)
        if mask.sum() > 0:
            per_class_acc[class_name] = (pred_classes[mask] == i).float().mean().item()
        else:
            per_class_acc[class_name] = 0.0
    
    results = {
        'model_name': model_name,
        'loss': avg_loss,
        'accuracy': accuracy,
        'per_class_accuracy': per_class_acc,
        'predictions': all_preds,
        'labels': all_labels
    }
    
    return results


print("✓ Comprehensive evaluation function defined!")


✓ Comprehensive evaluation function defined!


In [3]:
# Evaluate both Teacher and Student models on Test Set
print("=" * 70)
print("MODEL PERFORMANCE COMPARISON: TEACHER vs STUDENT")
print("=" * 70)

# Evaluate Teacher Model (PhaseNet)
print("\n📊 Evaluating Teacher Model (PhaseNet)...")
teacher_results = evaluate_model_comprehensive(test_loader, teacher_model, "PhaseNet", device)

print(f"\n👨‍🏫 Teacher Model (PhaseNet) Results:")
print(f"  Loss: {teacher_results['loss']:.6f}")
print(f"  Overall Accuracy: {teacher_results['accuracy']*100:.2f}%")
print(f"  Per-class Accuracy:")
for class_name, acc in teacher_results['per_class_accuracy'].items():
    print(f"    {class_name}: {acc*100:.2f}%")

# Evaluate Student Model (XiaoNet)
print("\n📊 Evaluating Student Model (XiaoNet)...")
student_results = evaluate_model_comprehensive(test_loader, student_model, "XiaoNet", device)

print(f"\n👨‍🎓 Student Model (XiaoNet) Results:")
print(f"  Loss: {student_results['loss']:.6f}")
print(f"  Overall Accuracy: {student_results['accuracy']*100:.2f}%")
print(f"  Per-class Accuracy:")
for class_name, acc in student_results['per_class_accuracy'].items():
    print(f"    {class_name}: {acc*100:.2f}%")

# Performance Comparison
print("\n" + "=" * 70)
print("📈 COMPARISON SUMMARY")
print("=" * 70)

# Model size comparison
teacher_params = sum(p.numel() for p in teacher_model.parameters())
student_params = sum(p.numel() for p in student_model.parameters())

print(f"\n🔢 Model Size:")
print(f"  Teacher: {teacher_params:,} parameters ({teacher_params * 4 / (1024**2):.2f} MB)")
print(f"  Student: {student_params:,} parameters ({student_params * 4 / (1024**2):.2f} MB)")
print(f"  Reduction: {(1 - student_params/teacher_params)*100:.1f}%")

print(f"\n📉 Loss Comparison:")
print(f"  Teacher Loss: {teacher_results['loss']:.6f}")
print(f"  Student Loss: {student_results['loss']:.6f}")
print(f"  Difference: {student_results['loss'] - teacher_results['loss']:.6f} ({((student_results['loss'] / teacher_results['loss']) - 1)*100:+.2f}%)")

print(f"\n🎯 Accuracy Comparison:")
print(f"  Teacher Accuracy: {teacher_results['accuracy']*100:.2f}%")
print(f"  Student Accuracy: {student_results['accuracy']*100:.2f}%")
print(f"  Difference: {(student_results['accuracy'] - teacher_results['accuracy'])*100:+.2f}%")

print(f"\n📊 Per-Class Accuracy Comparison:")
for class_name in ['P-wave', 'S-wave', 'Noise']:
    teacher_acc = teacher_results['per_class_accuracy'][class_name]
    student_acc = student_results['per_class_accuracy'][class_name]
    diff = (student_acc - teacher_acc) * 100
    print(f"  {class_name}:")
    print(f"    Teacher: {teacher_acc*100:.2f}%  |  Student: {student_acc*100:.2f}%  |  Diff: {diff:+.2f}%")

print("\n" + "=" * 70)
print("✓ Evaluation Complete!")
print("=" * 70)


MODEL PERFORMANCE COMPARISON: TEACHER vs STUDENT

📊 Evaluating Teacher Model (PhaseNet)...


NameError: name 'test_loader' is not defined

In [ ]:
# Visualize Predictions Comparison
import matplotlib.pyplot as plt

def plot_prediction_comparison(teacher_pred, student_pred, true_label, sample_idx=0):
    """
    Plot comparison of teacher and student predictions for a single sample.
    
    Args:
        teacher_pred: Teacher model predictions (batch, 3, time)
        student_pred: Student model predictions (batch, 3, time)
        true_label: Ground truth labels (batch, 3, time)
        sample_idx: Which sample from the batch to visualize
    """
    fig, axes = plt.subplots(4, 1, figsize=(15, 12))
    
    class_names = ['P-wave', 'S-wave', 'Noise']
    colors = ['red', 'blue', 'gray']
    
    # Plot ground truth
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        axes[0].plot(true_label[sample_idx, i, :].numpy(), 
                     label=class_name, color=color, alpha=0.7)
    axes[0].set_title('Ground Truth Labels', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Probability')
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([-0.05, 1.05])
    
    # Plot teacher predictions
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        axes[1].plot(teacher_pred[sample_idx, i, :].numpy(), 
                     label=class_name, color=color, alpha=0.7)
    axes[1].set_title('Teacher Model (PhaseNet) Predictions', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Probability')
    axes[1].legend(loc='upper right')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([-0.05, 1.05])
    
    # Plot student predictions
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        axes[2].plot(student_pred[sample_idx, i, :].numpy(), 
                     label=class_name, color=color, alpha=0.7)
    axes[2].set_title('Student Model (XiaoNet) Predictions', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Probability')
    axes[2].legend(loc='upper right')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_ylim([-0.05, 1.05])
    
    # Plot difference (Student - Teacher)
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        diff = student_pred[sample_idx, i, :].numpy() - teacher_pred[sample_idx, i, :].numpy()
        axes[3].plot(diff, label=class_name, color=color, alpha=0.7)
    axes[3].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[3].set_title('Difference (Student - Teacher)', fontsize=14, fontweight='bold')
    axes[3].set_ylabel('Probability Diff')
    axes[3].set_xlabel('Time Sample')
    axes[3].legend(loc='upper right')
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'prediction_comparison_sample_{sample_idx}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Saved comparison plot: prediction_comparison_sample_{sample_idx}.png")


# Plot first 3 samples
print("\n📊 Generating prediction comparison plots...")
for i in range(min(3, teacher_results['predictions'].shape[0])):
    plot_prediction_comparison(
        teacher_results['predictions'],
        student_results['predictions'],
        teacher_results['labels'],
        sample_idx=i
    )


In [ ]:
# Performance Efficiency Metrics
print("=" * 70)
print("⚡ EFFICIENCY METRICS")
print("=" * 70)

# Calculate inference time
import time

def measure_inference_time(model, dataloader, device, num_batches=10):
    """Measure average inference time per batch."""
    model.eval()
    times = []
    
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= num_batches:
                break
            
            X = batch["X"].to(device)
            
            start_time = time.time()
            _ = model(X)
            end_time = time.time()
            
            times.append(end_time - start_time)
    
    return np.mean(times), np.std(times)

print("\n⏱️ Measuring inference time...")

teacher_time, teacher_std = measure_inference_time(teacher_model, test_loader, device)
student_time, student_std = measure_inference_time(student_model, test_loader, device)

print(f"\nTeacher Model (PhaseNet):")
print(f"  Avg inference time: {teacher_time*1000:.2f} ± {teacher_std*1000:.2f} ms/batch")
print(f"  Samples per second: {batch_size / teacher_time:.1f}")

print(f"\nStudent Model (XiaoNet):")
print(f"  Avg inference time: {student_time*1000:.2f} ± {student_std*1000:.2f} ms/batch")
print(f"  Samples per second: {batch_size / student_time:.1f}")

speedup = teacher_time / student_time
print(f"\n🚀 Speedup: {speedup:.2f}x faster")

# Compression ratio
print(f"\n💾 Compression Metrics:")
print(f"  Parameter reduction: {(1 - student_params/teacher_params)*100:.1f}%")
print(f"  Size reduction: {teacher_params/student_params:.1f}x smaller")

# Efficiency score (accuracy per parameter)
teacher_efficiency = teacher_results['accuracy'] / (teacher_params / 1e6)
student_efficiency = student_results['accuracy'] / (student_params / 1e6)

print(f"\n📈 Efficiency (Accuracy per Million Parameters):")
print(f"  Teacher: {teacher_efficiency:.4f}")
print(f"  Student: {student_efficiency:.4f}")
print(f"  Student is {student_efficiency/teacher_efficiency:.2f}x more parameter-efficient")

print("\n" + "=" * 70)
print("✓ Efficiency analysis complete!")
print("=" * 70)
